In [17]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()#get(name="quick-starts-ws-139732")
exp = Experiment(workspace=ws, name="quick-starts-ws-139732")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-139732
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-139732


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

from azureml.exceptions import ComputeTargetException

cluster_name = "MyCluster"

try:
    comp_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    comp_target = ComputeTarget.create(ws, cluster_name, compute_config)

comp_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


# Run HyperDrive Config

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C": uniform(0.1, 5), "max_iter": choice(range(10, 200, 25))})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor = 0.2, delay_evaluation=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory = "./", entry_script="train.py", compute_target=comp_target)
est = ScriptRunConfig(source_directory = "./", script="train.py", compute_target=comp_target)



# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=12,
                             max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyper_run = exp.submit(hyperdrive_config)
RunDetails(hyper_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
hd_best_run = hyper_run.get_best_run_by_primary_metric()
#hd_best_run_metrics=hd_best_run.get_metrics()
hyper_run.get_children_sorted_by_primary_metric(top=1)
hd_best_run


In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [9]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
# Concat X & y-> no train test split is done because the model will use cross validation
data_cleaned = pd.concat([x,y], axis=1)

# Run AutoML Config

In [12]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_cleaned,
    label_column_name='y',
    n_cross_validations=5)

In [13]:
# Submit your automl run
automlrun = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_278b27a6-ec8c-4802-b23e-c4d079c2f580

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [14]:
# Retrieve and save your best automl model.
atml_best_run = automlrun.get_best_child()
atml_best_run.get_details()

{'runId': 'AutoML_278b27a6-ec8c-4802-b23e-c4d079c2f580_0',
 'status': 'Completed',
 'startTimeUtc': '2021-03-02T21:01:13.723129Z',
 'endTimeUtc': '2021-03-02T21:01:44.633466Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f',
  'pipeline_spec': '{"objects":[{"spec_class":"preproc","class_name":"MaxAbsScaler","module":"sklearn.preprocessing","param_args":[],"param_kwargs":{},"prepared_kwargs":{}},{"spec_class":"sklearn","class_name":"LightGBMClassifier","module":"automl.client.core.common.model_wrappers","param_args":[],"param_kwargs":{"min_data_in_leaf":20},"prepared_kwargs":{}}],"pipeline_id":"5dfac790c5c209f98a1da2dc1c7fb76f0397324f","module":"sklearn.pipeline","class_name":"Pipeline"}',
  'training_percent': '100',
  'predicted_cost': None,
  'iteration': '0',
  '_azureml.ComputeTargetType': 'local',
  '_aml_system_scenario_identification': 'Local.Child',
  'run_template': 'automl_child',
  'run_preprocessor': 'MaxAbsScaler'

In [15]:
# Close properly
comp_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

